In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import mlflow
import mlflow.pytorch
import numpy as np

Definir la red neuronal

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [ ]:
# Load the CSV file
df = pd.read_csv("fusion30.csv")

input_dim = 10
hidden_dim = 16
output_dim = 2
epochs = 10
learning_rate = 0.001

In [ ]:
# Start an MLflow run for experiment tracking
with mlflow.start_run("Toxic_Comments_Classification_Pytorch"):
    # Log hyperparameters
    mlflow.log_param("input_dim", input_dim)
    mlflow.log_param("hidden_dim", hidden_dim)
    mlflow.log_param("output_dim", output_dim)
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("learning_rate", learning_rate)

    # Model, loss, optimizer
    model = SimpleNN(input_dim, hidden_dim, output_dim)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        # Log loss for each epoch
        mlflow.log_metric("loss", loss.item(), step=epoch)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

    # Log the trained model
    mlflow.pytorch.log_model(model, "model")